In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('desag_1924.csv')

In [26]:
ipc = pd.read_csv('ipc_basic.csv')

In [29]:
df = df.merge(ipc[['Año', 'Indice_base_2025']], on='Año', how='inner')

In [32]:
df['apropiaciones_constantes_2025'] = (df['TOTAL'] / df['Indice_base_2025']) * 100

In [33]:
df.to_csv('desag_1924.csv', index=False)

In [51]:
import json
with open('dic_sec_ents.json') as f:
    dic_sec_ents = json.load(f)
with open('dic_sector.json') as f:
    dic_sector = json.load(f)

with open('dic_entities.json') as f:
    dic_entities = json.load(f)

In [43]:
df['Sector'] = df['Código de entidad'].astype(str).map(dic_sec_ents).astype(str).map(dic_sector)
df['Código de sector'] = df['Código de entidad'].astype(str).map(dic_sec_ents)

In [53]:
rev_ents = {}
del dic_entities["1719"]
for key, value in dic_entities.items():
    rev_ents[value] = key


In [54]:
df[df['Entidad'].str.contains('renovación')]['Entidad'].astype(str).map(rev_ents)

1183    1719
1184    1719
1185    1719
1186    1719
1187    1719
1188    1719
1189    1719
1190    1719
1191    1719
1192    1719
1193    1719
1194    1719
1195    1719
1196    1719
3462    1719
3463    1719
3464    1719
3465    1719
3466    1719
3467    1719
3468    1719
3469    1719
3470    1719
3471    1719
3472    1719
3473    1719
3474    1719
3475    1719
3476    1719
3477    1719
3478    1719
3479    1719
3480    1719
Name: Entidad, dtype: object